In [1]:
#pip install tensorflow transformers  tf-keras pandas
from transformers import pipeline
import pandas as pd
import os

2024-04-17 06:43:12.716554: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 06:43:12.724663: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 06:43:12.832126: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 06:43:14.083785: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#load model
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base", 
                      return_all_scores=True)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/329M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/home/ucloud/.local/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
#load data 
data_in = pd.read_csv("../in/GoT-scripts/Game_of_Thrones_Script.csv")

In [ ]:
data_out = pd.read_csv("../in/GoT-scripts/Game_of_Thrones_Script.csv")

In [33]:
#add emotion columns to copy of dataframe
data_out[["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]] = ""

In [34]:

for sent in range(len(data_in["Sentence"])):
    sentence = data_in["Sentence"][sent]
    emot_dicts = classifier(sentence)[0]
    scores = []
    for dict in emot_dicts:
        score = dict["score"]
        scores = scores + [score]
    data_out.loc[sent, "anger":"surprise"] = scores
data_out.to_csv("../out/emotion_scores.csv")
    

KeyboardInterrupt: 

In [23]:
ls = [] #empty list for the scores 
for dict in emot_scores[0]: #emot_scores[0] is the list of dicts
    score = (dict["score"]) #extract each score
    ls = ls + [score] #append to empty list 

data_out.loc[0, "anger":"suprise"] = ls #define row 0 and columns where the scores should be put in 

In [ ]:
##